In [2]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from ta.momentum import RSIIndicator
from ta.trend import ADXIndicator
from sklearn.linear_model import LinearRegression


from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest



from alpaca_secrets import APCA_API_KEY_ID, APCA_API_SECRET_KEY
import pandas as pd
import numpy as np
import talib

from backtesting import Strategy
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from datetime import datetime, timedelta
import inspect


trading_client = TradingClient(APCA_API_KEY_ID, APCA_API_SECRET_KEY)

import multiprocessing as mp

mp.set_start_method("fork", force=True)


# Data Collection

In [3]:
# search for US equities
search_params = GetAssetsRequest(asset_class=AssetClass.US_EQUITY)

assets = trading_client.get_all_assets(search_params)

In [4]:
def  prepare_data(list_symbol, n_years = 5):

    data_client = StockHistoricalDataClient(APCA_API_KEY_ID, APCA_API_SECRET_KEY)

    end_date = datetime(2025,7,15)
    start_date = end_date - timedelta(days=n_years*365)

    # Solicite os dados OHLC para os símbolos desejados
    bars_request = StockBarsRequest(
        symbol_or_symbols=list_symbol,
        timeframe=TimeFrame.Hour,
        start=start_date,
        end=end_date,
        adjustment="all"

    )

    bars = data_client.get_stock_bars(bars_request).data

    dfs = {}
    for sym in list_symbol:
        print(f"Processing {sym}")

        try:
            asset = trading_client.get_asset(sym)
            print(f"{asset.symbol}: Tradable = {asset.tradable}")
        except Exception as e:
            print(f"{sym}: Error - {e}")


        candle = bars.get(sym, None)
        if candle is not None:
            dfs[sym] = pd.DataFrame([{k: getattr(bar, k) for k in ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'vwap']} for bar in candle])
        
        
            # Supondo que df tenha ['open', 'high', 'low', 'close', 'volume']
            df = dfs[sym][['timestamp', 'open', 'high', 'low', 'close', 'volume']].copy()
            df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
            df['Timestamp'] = pd.to_datetime(df['Timestamp'])
            df.set_index('Timestamp', inplace=True)
            dfs[sym] = df
        
    return dfs


In [5]:
def prepare_crypto_data(list_symbol, n_years=1):
    client = CryptoHistoricalDataClient(APCA_API_KEY_ID, APCA_API_SECRET_KEY)
    end = datetime(2025, 7, 15)
    start = end - timedelta(days=n_years*365)

    crypto_data = {}

    for symbol in list_symbol:
        print(f"Fetching {symbol}...")

        request = CryptoBarsRequest(
            symbol_or_symbols=symbol,
            start=start,
            end=end,
            timeframe=TimeFrame.Hour,
            adjustment="all"
        )

        bars = client.get_crypto_bars(request).df
        if bars.empty:
            print(f"No data for {symbol}")
            continue

        df = bars[bars.index.get_level_values(0) == symbol].droplevel(0).copy()
        df.index.name = "timestamp"
        df = df.rename(columns=str.lower)
        df = df.reset_index()

        # Supondo que df tenha ['open', 'high', 'low', 'close', 'volume']
        df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']].copy()
        df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df.set_index('Timestamp', inplace=True)

        crypto_data[symbol] = df

    return crypto_data


## ETFs Data

•	ETFs de renda fixa (ex: TLT, HYG) e ouro (GLD, IAU) costumam ter movimentos mais suaves, o que pode ajudar a estratégia.


In [6]:
list_symbol_ = ["SPY","QQQ","IWM","DIA","XLF","XLK","GLD","IAU","TLT","HYG",]

etfs_close_data = prepare_data(list_symbol_, n_years = 1)

etfs_close_data.keys()

Processing SPY
SPY: Tradable = True
Processing QQQ
QQQ: Tradable = True
Processing IWM
IWM: Tradable = True
Processing DIA
DIA: Tradable = True
Processing XLF
XLF: Tradable = True
Processing XLK
XLK: Tradable = True
Processing GLD
GLD: Tradable = True
Processing IAU
IAU: Tradable = True
Processing TLT
TLT: Tradable = True
Processing HYG
HYG: Tradable = True


dict_keys(['SPY', 'QQQ', 'IWM', 'DIA', 'XLF', 'XLK', 'GLD', 'IAU', 'TLT', 'HYG'])

## Equities Data

•	Ações com volatilidade moderada (ex: MSFT, JPM) também funcionam bem para testes de reversão à média.


In [7]:
eqt_symbol_ = ["AAPL","MSFT","GOOG","META","TSLA","NVDA","JP",]

eqt_close_data = prepare_data(eqt_symbol_, n_years = 1)

eqt_close_data.keys()

Processing AAPL
AAPL: Tradable = True
Processing MSFT
MSFT: Tradable = True
Processing GOOG
GOOG: Tradable = True
Processing META
META: Tradable = True
Processing TSLA
TSLA: Tradable = True
Processing NVDA
NVDA: Tradable = True
Processing JP
JP: Error - {"code":40410000,"message":"asset not found for JP"}


dict_keys(['AAPL', 'MSFT', 'GOOG', 'META', 'TSLA', 'NVDA'])

## Crypto Data


In [8]:
crypto_symbols = ["BTC/USD", "ETH/USD", "SOL/USD"]

crypto_close_data = prepare_crypto_data(crypto_symbols, n_years=1)

# Exemplo de acesso:
crypto_close_data.keys()  # dict com os DataFrames
crypto_close_data["BTC/USD"].head()

Fetching BTC/USD...
Fetching ETH/USD...
Fetching SOL/USD...


,Open,High,Low,Close,Volume
Timestamp,,,,,
2024-07-15 00:00:00+00:00,60856.4680,61333.4210,60694.5350,61229.6715,0.000095
2024-07-15 01:00:00+00:00,61269.1075,61825.9850,61237.4785,61429.5710,0.003157
2024-07-15 02:00:00+00:00,61386.0000,62622.5890,61318.6900,62546.4970,0.003873
2024-07-15 03:00:00+00:00,62519.0490,62798.4595,62370.4200,62693.7200,0.177868
2024-07-15 04:00:00+00:00,62728.5700,62842.1500,62473.0100,62623.6600,0.033972


# Trend strats

## MACrossover

In [9]:
class MACrossover(Strategy):
    short_window = 5
    long_window = 60
    buffer_pct = 0.01  # 0.1%

    def init(self):
        close = self.data.Close
        self.ma_short = self.I(lambda x: pd.Series(x).rolling(self.short_window).mean(), close, name='ma_short')
        self.ma_long = self.I(lambda x: pd.Series(x).rolling(self.long_window).mean(), close, name='ma_long')

    def next(self):
        if len(self.ma_short) < 2 or len(self.ma_long) < 2:
            return

        price = self.data.Close[-1]
        diff = self.ma_short[-1] - self.ma_long[-1]

        # Aplica o buffer
        if diff > self.buffer_pct * price and self.ma_short[-2] <= self.ma_long[-2]:
            if self.position.is_short:
                self.position.close()
            if not self.position.is_long:
                self.buy(size=10)

        elif diff < -self.buffer_pct * price and self.ma_short[-2] >= self.ma_long[-2]:
            if self.position.is_long:
                self.position.close()
            if not self.position.is_short:
                self.sell(size=10)

In [30]:
class MACrossoverADX(Strategy):
    short_window = 23
    long_window = 40
    adx_threshold = 35
    T_period = 14

    def init(self):
        close = self.data.Close

        self.ma_short = self.I(lambda x: pd.Series(x).rolling(self.short_window).mean(), close, name='ma_short')
        self.ma_long = self.I(lambda x: pd.Series(x).rolling(self.long_window).mean(), close, name='ma_long')

        self.adx = self.I(lambda h, l, c: talib.ADX(h, l, c, timeperiod=self.T_period),
                          self.data.High, self.data.Low, self.data.Close,
                          name='adx')

    def next(self):
        if len(self.ma_short) < 2 or len(self.ma_long) < 2 or len(self.adx) < 2:
            return

        price = self.data.Close[-1]
        adx = self.adx[-1]

        # Só entra se ADX estiver acima do limite mínimo
        if adx < self.adx_threshold:
            return

        # Lógica de cruzamento das médias
        cross_up = self.ma_short[-1] > self.ma_long[-1] and self.ma_short[-2] <= self.ma_long[-2]
        cross_down = self.ma_short[-1] < self.ma_long[-1] and self.ma_short[-2] >= self.ma_long[-2]

        if cross_up:
            if self.position.is_short:
                self.position.close()
            if not self.position.is_long:
                self.buy(size=10)

        elif cross_down:
            if self.position.is_long:
                self.position.close()
            if not self.position.is_short:
                self.sell(size=10)

In [39]:
class MACrossoverADXStopLoss(Strategy):
    short_window = 25
    long_window = 40
    adx_threshold = 30
    T_period = 14
    stop_loss_pct = 0.01  # 1%

    def init(self):
        close = self.data.Close

        self.ma_short = self.I(lambda x: pd.Series(x).rolling(self.short_window).mean(), close, name='ma_short')
        self.ma_long = self.I(lambda x: pd.Series(x).rolling(self.long_window).mean(), close, name='ma_long')

        self.adx = self.I(lambda h, l, c: talib.ADX(h, l, c, timeperiod=self.T_period),
                          self.data.High, self.data.Low, self.data.Close,
                          name='adx')

    def next(self):
        if len(self.ma_short) < 2 or len(self.ma_long) < 2 or len(self.adx) < 2:
            return

        price = self.data.Close[-1]
        adx = self.adx[-1]

        # Stop loss dinâmico
        if self.position and self.trades:
            entry_price = self.trades[-1].entry_price

            if self.position.is_long:
                loss = (price - entry_price) / entry_price
                if loss < -self.stop_loss_pct:
                    self.position.close()
                    return

            elif self.position.is_short:
                loss = (entry_price - price) / entry_price
                if loss < -self.stop_loss_pct:
                    self.position.close()
                    return

        if adx < self.adx_threshold:
            return

        cross_up = self.ma_short[-1] > self.ma_long[-1] and self.ma_short[-2] <= self.ma_long[-2]
        cross_down = self.ma_short[-1] < self.ma_long[-1] and self.ma_short[-2] >= self.ma_long[-2]

        if cross_up:
            if self.position.is_short:
                self.position.close()
            if not self.position.is_long:
                self.buy(size=10)

        elif cross_down:
            if self.position.is_long:
                self.position.close()
            if not self.position.is_short:
                self.sell(size=10)

In [10]:
class MACrossoverADXTrendAware(Strategy):
    short_window = 20
    long_window = 50
    adx_threshold = 20
    T_period = short_window


    def init(self):
        close = self.data.Close

        self.ma_short = self.I(lambda x: pd.Series(x).rolling(self.short_window).mean(), close, name='ma_short')
        self.ma_long = self.I(lambda x: pd.Series(x).rolling(self.long_window).mean(), close, name='ma_long')

        self.adx = self.I(lambda h, l, c: talib.ADX(h, l, c, timeperiod=self.T_period),
                          self.data.High, self.data.Low, self.data.Close,
                          name='adx')

    def next(self):
        if len(self.ma_short) < 2 or len(self.ma_long) < 2 or len(self.adx) < 2:
            return

        price = self.data.Close[-1]
        adx = self.adx[-1]

        cross_up = self.ma_short[-1] > self.ma_long[-1] and self.ma_short[-2] <= self.ma_long[-2]
        cross_down = self.ma_short[-1] < self.ma_long[-1] and self.ma_short[-2] >= self.ma_long[-2]

        # ENTRY: cruzamento com tendência forte
        if not self.position:
            if cross_up and adx > self.adx_threshold:
                self.buy(size=10)
            elif cross_down and adx > self.adx_threshold:
                self.sell(size=10)

        # EXIT: tendência perdeu força ou cruzamento contrário
        else:
            exit_condition = adx < self.adx_threshold

            if self.position.is_long:
                if exit_condition or cross_down:
                    self.position.close()

            elif self.position.is_short:
                if exit_condition or cross_up:
                    self.position.close()

In [11]:
class MACrossoverADXBuffered(Strategy):
    short_window = 20
    long_window = 50
    adx_threshold = 20
    buffer_pct = 0.001  # 1% do preço atual

    def init(self):
        close = self.data.Close

        self.ma_short = self.I(lambda x: pd.Series(x).rolling(self.short_window).mean(), close, name='ma_short')
        self.ma_long = self.I(lambda x: pd.Series(x).rolling(self.long_window).mean(), close, name='ma_long')

        self.adx = self.I(lambda h, l, c: talib.ADX(h, l, c, timeperiod=14),
                          self.data.High, self.data.Low, self.data.Close,
                          name='adx')

    def next(self):
        if len(self.ma_short) < 2 or len(self.ma_long) < 2 or len(self.adx) < 2:
            return

        price = self.data.Close[-1]
        adx = self.adx[-1]
        diff = self.ma_short[-1] - self.ma_long[-1]
        buffer = self.buffer_pct * price

        cross_up = diff > buffer and self.ma_short[-2] <= self.ma_long[-2]
        cross_down = diff < -buffer and self.ma_short[-2] >= self.ma_long[-2]

        # ENTRY: cruzamento com ADX forte e diferença significativa
        if not self.position:
            if cross_up and adx > self.adx_threshold:
                self.buy(size=10)
            elif cross_down and adx > self.adx_threshold:
                self.sell(size=10)

        # EXIT: tendência perdeu força ou cruzamento oposto com margem suficiente
        else:
            exit_due_to_adx = adx < self.adx_threshold
            reverse_cross_up = diff > buffer and self.ma_short[-2] <= self.ma_long[-2]
            reverse_cross_down = diff < -buffer and self.ma_short[-2] >= self.ma_long[-2]

            if self.position.is_long and (exit_due_to_adx or reverse_cross_down):
                self.position.close()

            elif self.position.is_short and (exit_due_to_adx or reverse_cross_up):
                self.position.close()

In [12]:

class TripleMACrossoverADXBuffered(Strategy):
    # Atributos de classe (podem ser sobrescritos externamente)
    short_window = 5
    mid_window = 40
    long_window = 120
    adx_threshold = 35
    buffer_pct = 0.0005

    def init(self):
        close = self.data.Close

        self.ma_short = self.I(lambda x: pd.Series(x).rolling(self.short_window).mean(), close, name='ma_short')
        self.ma_mid = self.I(lambda x: pd.Series(x).rolling(self.mid_window).mean(), close, name='ma_mid')
        self.ma_long = self.I(lambda x: pd.Series(x).rolling(self.long_window).mean(), close, name='ma_long')

        self.adx = self.I(lambda h, l, c: talib.ADX(h, l, c, timeperiod=14),
                          self.data.High, self.data.Low, self.data.Close,
                          name='adx')

    def next(self):
        if len(self.ma_short) < 2 or len(self.ma_mid) < 2 or len(self.ma_long) < 2 or len(self.adx) < 2:
            return

        price = self.data.Close[-1]
        adx = self.adx[-1]
        buffer = self.buffer_pct * price

        s, m, l = self.ma_short[-1], self.ma_mid[-1], self.ma_long[-1]
        s_prev, m_prev, l_prev = self.ma_short[-2], self.ma_mid[-2], self.ma_long[-2]

        aligned_up = s > m + buffer and m > l + buffer
        aligned_down = s < m - buffer and m < l - buffer

        was_aligned_up = s_prev > m_prev and m_prev > l_prev
        was_aligned_down = s_prev < m_prev and m_prev < l_prev

        if not self.position and adx > self.adx_threshold:
            if aligned_up:
                self.buy(size=10)
            elif aligned_down:
                self.sell(size=10)

        elif self.position:
            not_aligned = not (aligned_up or aligned_down)
            adx_weak = adx < self.adx_threshold

            if self.position.is_long and (not_aligned or adx_weak):
                self.position.close()
            elif self.position.is_short and (not_aligned or adx_weak):
                self.position.close()

In [13]:

class TripleMACrossoverTrendOnly(Strategy):
    # Parâmetros fixos (sem externalização)
    short_window = 25
    mid_window = 70
    long_window = 160
    buffer_pct = 0.0  # 2%

    def init(self):
        close = self.data.Close

        self.ma_short = self.I(lambda x: pd.Series(x).rolling(self.short_window).mean(), close, name='ma_short')
        self.ma_mid = self.I(lambda x: pd.Series(x).rolling(self.mid_window).mean(), close, name='ma_mid')
        self.ma_long = self.I(lambda x: pd.Series(x).rolling(self.long_window).mean(), close, name='ma_long')

    def next(self):
        if len(self.ma_short) < 2 or len(self.ma_mid) < 2 or len(self.ma_long) < 2:
            return

        price = self.data.Close[-1]
        buffer = self.buffer_pct * price

        s, m, l = self.ma_short[-1], self.ma_mid[-1], self.ma_long[-1]
        s_prev, m_prev, l_prev = self.ma_short[-2], self.ma_mid[-2], self.ma_long[-2]

        # Alinhamentos com margem
        aligned_up = s > m + buffer and m > l + buffer
        aligned_down = s < m - buffer and m < l - buffer

        was_aligned_up = s_prev > m_prev and m_prev > l_prev
        was_aligned_down = s_prev < m_prev and m_prev < l_prev

        # ENTRY
        if not self.position:
            if aligned_up:
                self.buy(size=10)
            elif aligned_down:
                self.sell(size=10)

        # EXIT
        elif self.position:
            trend_broken = not (aligned_up or aligned_down)

            if self.position.is_long and trend_broken:
                self.position.close()
            elif self.position.is_short and trend_broken:
                self.position.close()

In [ ]:
class TripleMACrossoverBullishBias(Strategy):
    short_window = 5
    mid_window = 50
    long_window = 100

    buffer_up = 0.05   # 1% para entradas compradas
    buffer_down = 0.1 # 3% para entradas vendidas

    def init(self):
        close = self.data.Close

        self.ma_short = self.I(lambda x: pd.Series(x).rolling(self.short_window).mean(), close, name='ma_short')
        self.ma_mid = self.I(lambda x: pd.Series(x).rolling(self.mid_window).mean(), close, name='ma_mid')
        self.ma_long = self.I(lambda x: pd.Series(x).rolling(self.long_window).mean(), close, name='ma_long')

    def next(self):
        if len(self.ma_short) < 2 or len(self.ma_mid) < 2 or len(self.ma_long) < 2:
            return

        price = self.data.Close[-1]
        s, m, l = self.ma_short[-1], self.ma_mid[-1], self.ma_long[-1]
        s_prev, m_prev, l_prev = self.ma_short[-2], self.ma_mid[-2], self.ma_long[-2]

        # Buffers específicos por direção
        buffer_long = self.buffer_up * price
        buffer_short = self.buffer_down * price

        # Alinhamento com viés de alta (long = mais fácil)
        aligned_up = s > m + buffer_long and m > l + buffer_long
        aligned_down = s < m - buffer_short and m < l - buffer_short

        was_aligned_up = s_prev > m_prev and m_prev > l_prev
        was_aligned_down = s_prev < m_prev and m_prev < l_prev

        # ENTRY com viés
        if not self.position:
            if aligned_up:
                self.buy(size=10)
            elif aligned_down:
                self.sell(size=10)

        # EXIT se desalinhamento
        elif self.position:
            still_aligned = aligned_up if self.position.is_long else aligned_down

            if not still_aligned:
                self.position.close()

## CryptoMomentumADX

In [19]:

class CryptoMomentumADX(Strategy):
    def init(self):
        close = self.data.Close

        self.sma50 = self.I(lambda x: talib.SMA(x, timeperiod=50), close, name='sma50')
        self.ema7 = self.I(lambda x: talib.EMA(x, timeperiod=7), close, name='ema7')
        self.rsi2 = self.I(lambda x: talib.RSI(x, timeperiod=2), close, name='rsi2')
        self.adx2 = self.I(lambda h, l, c: talib.ADX(h, l, c, timeperiod=2), 
                           self.data.High, self.data.Low, close, name='adx2')

    def next(self):
        if len(self.data) < 50:
            return  # garante janela mínima para SMA(50)

        price = self.data.Close[-1]
        sma = self.sma50[-1]
        ema = self.ema7[-1]
        rsi = self.rsi2[-1]
        adx = self.adx2[-1]

        if not self.position:
            if price > sma and price > ema and rsi > adx:
                self.buy(size=10)

        elif rsi < adx:
            self.position.close()

# Mean Reversion Strat

## ZScoreMeanReversion

In [20]:
class ZScoreMeanReversion(Strategy):
    window = 20
    threshold = 2

    def init(self):
        close = self.data.Close
        self.ma = self.I(lambda x: pd.Series(x).rolling(self.window).mean(), close, name='ma')
        self.std = self.I(lambda x: pd.Series(x).rolling(self.window).std(), close, name='std')
        self.z = self.I(lambda x, ma, std: (x - ma) / std, close, self.ma, self.std, name='z')

    def next(self):
        if len(self.z) < 2:
            return

        z = self.z[-1]

        if not self.position:
            if z < -self.threshold:
                self.buy(size=10)
            elif z > self.threshold:
                self.sell(size=10)
        else:
            if self.position.is_long and z > 0:
                self.position.close()
            elif self.position.is_short and z < 0:
                self.position.close()

## ZScoreReturnReversion

In [21]:
class ZScoreReturnReversion(Strategy):
    external_z = None  # Deve ser definido antes do backtest

    def init(self):
        if self.external_z is None:
            raise ValueError("ZScoreReturnReversion precisa de 'external_z' definido.")
        self.z = self.I(lambda x: self.external_z.loc[pd.Index(x.index)], self.data.df, name='z_ext')

    def next(self):
        if len(self.z) < 2:
            return

        z = self.z[-1]

        if not self.position:
            if z < -2:
                self.buy(size=10)
            elif z > 2:
                self.sell(size=10)
        else:
            if self.position.is_long and z > 0:
                self.position.close()
            elif self.position.is_short and z < 0:
                self.position.close()

## RSIMeanReversion

In [22]:
class RSIMeanReversion(Strategy):
    rsi_period = 14

    def init(self):
        self.rsi = self.I(lambda x: talib.RSI(x, timeperiod=self.rsi_period), self.data.Close, name='rsi')

    def next(self):
        if len(self.rsi) < 2:
            return

        rsi = self.rsi[-1]

        if not self.position:
            if rsi < 30:
                self.buy(size=10)
            elif rsi > 70:
                self.sell(size=10)
        else:
            if self.position.is_long and rsi > 45:
                self.position.close()
            elif self.position.is_short and rsi < 55:
                self.position.close()

## KeltnerChannelReversion

In [23]:
class KeltnerChannelReversion(Strategy):
    atr_period = 20

    def init(self):
        high = self.data.High
        low = self.data.Low
        close = self.data.Close
        self.atr = self.I(lambda h, l, c: talib.ATR(h, l, c, timeperiod=self.atr_period), high, low, close, name='atr')
        self.ma = self.I(lambda x: talib.EMA(x, timeperiod=self.atr_period), close, name='ma')
        self.upper = self.I(lambda ma, atr: ma + 2 * atr, self.ma, self.atr, name='upper')
        self.lower = self.I(lambda ma, atr: ma - 2 * atr, self.ma, self.atr, name='lower')

    def next(self):
        price = self.data.Close[-1]
        if len(self.upper) < 2 or len(self.lower) < 2:
            return

        if not self.position:
            if price < self.lower[-1]:
                self.buy(size=10)
            elif price > self.upper[-1]:
                self.sell(size=10)
        else:
            if self.position.is_long and price >= self.ma[-1]:
                self.position.close()
            elif self.position.is_short and price <= self.ma[-1]:
                self.position.close()

## GapReversal

In [24]:
class GapReversal(Strategy):
    gap_thresh = 0.02

    def init(self):
        self.recent_mean = self.I(lambda x: pd.Series(x).rolling(5).mean(), self.data.Close, name='mean5')

    def next(self):
        if len(self.data) < 2:
            return

        open_price = self.data.Open[-1]
        prev_close = self.data.Close[-2]
        gap = (open_price - prev_close) / prev_close

        price = self.data.Close[-1]
        mean = self.recent_mean[-1]

        if not self.position:
            if gap < -self.gap_thresh:
                self.buy(size=10)
            elif gap > self.gap_thresh:
                self.sell(size=10)
        else:
            if self.position.is_long and price >= mean:
                self.position.close()
            elif self.position.is_short and price <= mean:
                self.position.close()

## MeanReversionIBS

In [46]:

class MeanReversionIBS(Strategy):
    hl_window = 25
    high_window = 10
    ibs_threshold = 0.3

    def init(self):
        high = self.data.High
        low = self.data.Low
        close = self.data.Close

        self.hl_range = self.I(lambda h, l: h - l, high, low)
        self.mean_hl = self.I(lambda x: pd.Series(x).rolling(self.hl_window).mean(), self.hl_range)
        self.rolling_high = self.I(lambda x: pd.Series(x).rolling(self.high_window).max(), high)

        # Lower band
        self.lower_band = self.I(lambda rh, mhl: rh - 2.5 * mhl, self.rolling_high, self.mean_hl)

        # IBS
        self.ibs = self.I(lambda c, l, h: (c - l) / (h - l + 1e-8), close, low, high)  # evita divisão por zero

    def next(self):
        if len(self.ibs) < 2 or len(self.lower_band) < 2:
            return

        close = self.data.Close[-1]
        high_yesterday = self.data.High[-2]
        ibs = self.ibs[-1]
        lower_band = self.lower_band[-1]

        # Saída: se o preço fechar acima da máxima de ontem
        if self.position:
            if close > high_yesterday:
                self.position.close()
                return

        # Entrada
        if close < lower_band and ibs < self.ibs_threshold:
            if not self.position.is_long:
                self.buy(size=10)

# Backtesting

In [25]:
print(list(etfs_close_data.keys()))

print(eqt_close_data.keys())

['SPY', 'QQQ', 'IWM', 'DIA', 'XLF', 'XLK', 'GLD', 'IAU', 'TLT', 'HYG']
dict_keys(['AAPL', 'MSFT', 'GOOG', 'META', 'TSLA', 'NVDA'])


In [26]:

def run_strategies(df, strategies, plots=False):
    df_bt = df.reset_index()[['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']].copy()
    df_bt['Timestamp'] = pd.to_datetime(df_bt['Timestamp'])
    df_bt.set_index('Timestamp', inplace=True)

    # Pré-calcula z-score histórico de retornos (se necessário)
    zscore = None
    for strategy in strategies:
        if hasattr(strategy, 'external_z'):
            if zscore is None:
                zscore = compute_historical_zscore(df_bt['Close'])

            strategy.external_z = zscore

    for strategy in strategies:
        print("="*60)
        print(f"Running strategy: {strategy.__name__}")
        print("-"*60)
        
        bt = Backtest(df_bt, strategy, cash=100_000, commission=.01, exclusive_orders=True)
        results = bt.run()

        if plots:
            bt.plot()

        print(f"Return [%]:           {results['Return [%]']:.2f}")
        print(f"Buy & Hold Return [%]: {results['Buy & Hold Return [%]']:.2f}")
        print(f"Sharpe Ratio:         {results['Sharpe Ratio']:.2f}")
        print(f"# Trades:             {results['_trades'].shape[0]}")
        print(f"Win Rate:             {results['Win Rate [%]']:.2f}%")
        print(f"Max Drawdown [%]:     {results['Max. Drawdown [%]']:.2f}")
        print(f"Avg Trade Duration:   {results['Avg. Trade Duration']}")
        print(f"Best Trade [%]:       {results['Best Trade [%]']:.2f}")
        print(f"Worst Trade [%]:      {results['Worst Trade [%]']:.2f}")
        print("="*60)

In [28]:
run_strategies(etfs_close_data["TLT"], [MACrossover, DonchianBreakout, ADXTrend, EMA_RSI_Filter, LinearSlope], plots=False)

Running strategy: MACrossover
------------------------------------------------------------
Return [%]:           0.00
Buy & Hold Return [%]: -3.49
Sharpe Ratio:         nan
# Trades:             0
Win Rate:             nan%
Max Drawdown [%]:     -0.00
Avg Trade Duration:   nan
Best Trade [%]:       nan
Worst Trade [%]:      nan
Running strategy: DonchianBreakout
------------------------------------------------------------
Return [%]:           -0.57
Buy & Hold Return [%]: -3.49
Sharpe Ratio:         -3.13
# Trades:             23
Win Rate:             39.13%
Max Drawdown [%]:     -0.62
Avg Trade Duration:   15 days 07:00:00
Best Trade [%]:       3.67
Worst Trade [%]:      -4.93
Running strategy: ADXTrend
------------------------------------------------------------
Return [%]:           -1.38
Buy & Hold Return [%]: -4.58
Sharpe Ratio:         -6.66
# Trades:             80
Win Rate:             37.50%
Max Drawdown [%]:     -1.38
Avg Trade Duration:   1 days 04:00:00
Best Trade [%]:     

In [32]:
etfs_close_data.keys()

dict_keys(['SPY', 'QQQ', 'IWM', 'DIA', 'XLF', 'XLK', 'GLD', 'IAU', 'TLT', 'HYG'])

In [33]:
eqt_close_data.keys()

dict_keys(['AAPL', 'MSFT', 'GOOG', 'META', 'TSLA', 'NVDA'])

In [43]:
df = eqt_close_data["META"]

bt = Backtest(df, MACrossover, cash=100_000, commission=.01, exclusive_orders=True)
results = bt.run()    
# results = bt.optimize(
#     short_window=range(5, 30, 2),
#     long_window=range(30, 100, 5),
#     buffer_pct=[0.001, 0.005, 0.01, 0.02],
#     maximize='Sharpe Ratio',
#     constraint=lambda p: p.short_window < p.long_window
# )
print(results._strategy)

print(f"Return [%]:           {results['Return [%]']:.2f}")
print(f"Buy & Hold Return [%]: {results['Buy & Hold Return [%]']:.2f}")
print(f"Sharpe Ratio:         {results['Sharpe Ratio']:.2f}")
print(f"# Trades:             {results['_trades'].shape[0]}")
print(f"Win Rate:             {results['Win Rate [%]']:.2f}%")
print(f"Max Drawdown [%]:     {results['Max. Drawdown [%]']:.2f}")
print(f"Avg Trade Duration:   {results['Avg. Trade Duration']}")
print(f"Best Trade [%]:       {results['Best Trade [%]']:.2f}")
print(f"Worst Trade [%]:      {results['Worst Trade [%]']:.2f}")
print("="*60)
bt.plot()

MACrossover
Return [%]:           1.52
Buy & Hold Return [%]: 51.99
Sharpe Ratio:         1.18
# Trades:             0
Win Rate:             nan%
Max Drawdown [%]:     -1.06
Avg Trade Duration:   nan
Best Trade [%]:       nan
Worst Trade [%]:      nan


GridPlot(id='p9847', ...)

In [16]:
results._strategy

<Strategy MACrossover>

In [44]:
# na tesla
# <Strategy MACrossover(short_window=17,long_window=80,buffer_pct=0.002)>
# <Strategy MACrossover(short_window=5,long_window=60,buffer_pct=0.01)>

# na apple
# <Strategy MACrossover(short_window=5,long_window=60,buffer_pct=0.01)>

# na nvidia
# <Strategy MACrossover(short_window=5,long_window=60,buffer_pct=0.01)>

# na META
# <Strategy MACrossover(short_window=5,long_window=60,buffer_pct=0.01)>

# no XLF
# <Strategy MACrossover(short_window=5,long_window=60,buffer_pct=0.01)>


# o SPY
# <Strategy MACrossover(short_window=5,long_window=60,buffer_pct=0.01)>


In [31]:
bt = Backtest(df, MACrossoverADX, cash=100_000, commission=.01, exclusive_orders=True)
results = bt.run()
# results = bt.optimize(
#     short_window=range(5, 30, 2),
#     long_window=range(30, 100, 5),
#     adx_threshold=[20, 25, 30, 35],
#     # buffer_pct=[0.000, 0.001, 0.002],
#     maximize='Sharpe Ratio',
#     constraint=lambda p: p.short_window < p.long_window
# )
print(results._strategy)
# MACrossoverADX(short_window=23,long_window=40,adx_threshold=35)
# MACrossoverADX(short_window=11,long_window=70,adx_threshold=20)
# MACrossoverADX(short_window=7,long_window=85,adx_threshold=30) 4.16


print(f"Return [%]:           {results['Return [%]']:.2f}")
print(f"Buy & Hold Return [%]: {results['Buy & Hold Return [%]']:.2f}")
print(f"Sharpe Ratio:         {results['Sharpe Ratio']:.2f}")
print(f"# Trades:             {results['_trades'].shape[0]}")
print(f"Win Rate:             {results['Win Rate [%]']:.2f}%")
print(f"Max Drawdown [%]:     {results['Max. Drawdown [%]']:.2f}")
print(f"Avg Trade Duration:   {results['Avg. Trade Duration']}")
print(f"Best Trade [%]:       {results['Best Trade [%]']:.2f}")
print(f"Worst Trade [%]:      {results['Worst Trade [%]']:.2f}")
print("="*60)
bt.plot()

MACrossoverADX
Return [%]:           1.06
Buy & Hold Return [%]: 27.28
Sharpe Ratio:         0.49
# Trades:             1
Win Rate:             100.00%
Max Drawdown [%]:     -2.63
Avg Trade Duration:   22 days 12:00:00
Best Trade [%]:       5.14
Worst Trade [%]:      5.14


/Users/guilistocco/Trabalho_Final/trading_strategy/trading_env/lib/python3.10/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p7312', ...)

In [33]:
results._strategy

# na tesla
# MACrossoverADX(short_window=23,long_window=40,adx_threshold=35)
# MACrossoverADX(short_window=11,long_window=70,adx_threshold=20)
# MACrossoverADX(short_window=7,long_window=85,adx_threshold=30) 4.16

<Strategy MACrossoverADX>

In [ ]:
bt = Backtest(df, MACrossoverADXStopLoss, cash=100_000, commission=.01, exclusive_orders=True)
# results = bt.run()
results = bt.optimize(
    short_window=range(5, 30, 2),
    long_window=range(30, 100, 5),
    adx_threshold=[20, 25, 30, 35],
    stop_loss_pct=[0.000, 0.01, 0.05],
    maximize='Sharpe Ratio',
    constraint=lambda p: p.short_window < p.long_window
)
print(results._strategy)
# na tesla
# MACrossoverADXStopLoss(short_window=25,long_window=40,adx_threshold=30) 2.61
# MACrossoverADXStopLoss(short_window=17,long_window=75,adx_threshold=35,stop_loss_pct=0.05) 3.01

# na microsoft
# MACrossoverADXStopLoss(short_window=13,long_window=60,adx_threshold=25,stop_loss_pct=0.01)

# na META
# MACrossoverADXStopLoss(short_window=13,long_window=45,adx_threshold=25,stop_loss_pct=0.05)



print(f"Return [%]:           {results['Return [%]']:.2f}")
print(f"Buy & Hold Return [%]: {results['Buy & Hold Return [%]']:.2f}")
print(f"Sharpe Ratio:         {results['Sharpe Ratio']:.2f}")
print(f"# Trades:             {results['_trades'].shape[0]}")
print(f"Win Rate:             {results['Win Rate [%]']:.2f}%")
print(f"Max Drawdown [%]:     {results['Max. Drawdown [%]']:.2f}")
print(f"Avg Trade Duration:   {results['Avg. Trade Duration']}")
print(f"Best Trade [%]:       {results['Best Trade [%]']:.2f}")
print(f"Worst Trade [%]:      {results['Worst Trade [%]']:.2f}")
print("="*60)
bt.plot()

/Users/guilistocco/Trabalho_Final/trading_strategy/trading_env/lib/python3.10/site-packages/backtesting/backtesting.py:1606: UserWarning: Searching for best of 2184 configurations.
  output = _optimize_grid()


MACrossoverADXStopLoss(short_window=13,long_window=45,adx_threshold=25,stop_loss_pct=0.05)
Return [%]:           4.51
Buy & Hold Return [%]: 56.37
Sharpe Ratio:         1.79
# Trades:             13
Win Rate:             61.54%
Max Drawdown [%]:     -1.52
Avg Trade Duration:   26 days 23:00:00
Best Trade [%]:       27.44
Worst Trade [%]:      -5.25


/Users/guilistocco/Trabalho_Final/trading_strategy/trading_env/lib/python3.10/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p10690', ...)

In [48]:
bt = Backtest(df, MeanReversionIBS, cash=100_000, commission=.01, exclusive_orders=True)
results = bt.run()
# results = bt.optimize(
#     hl_window=range(5, 30, 2),
#     high_window=range(30, 100, 5),
#     ibs_threshold=[0.1, 0.2, 0.3, 0.4],
#     maximize='Sharpe Ratio',
#     constraint=lambda p: p.hl_window < p.high_window
# )
print(results._strategy)


print(f"Return [%]:           {results['Return [%]']:.2f}")
print(f"Buy & Hold Return [%]: {results['Buy & Hold Return [%]']:.2f}")
print(f"Sharpe Ratio:         {results['Sharpe Ratio']:.2f}")
print(f"# Trades:             {results['_trades'].shape[0]}")
print(f"Win Rate:             {results['Win Rate [%]']:.2f}%")
print(f"Max Drawdown [%]:     {results['Max. Drawdown [%]']:.2f}")
print(f"Avg Trade Duration:   {results['Avg. Trade Duration']}")
print(f"Best Trade [%]:       {results['Best Trade [%]']:.2f}")
print(f"Worst Trade [%]:      {results['Worst Trade [%]']:.2f}")
print("="*60)
bt.plot()

/Users/guilistocco/Trabalho_Final/trading_strategy/trading_env/lib/python3.10/site-packages/backtesting/backtesting.py:1606: UserWarning: Searching for best of 728 configurations.
  output = _optimize_grid()


MeanReversionIBS(hl_window=13,high_window=30,ibs_threshold=0.1)
Return [%]:           -27.37
Buy & Hold Return [%]: 48.58
Sharpe Ratio:         -14.77
# Trades:             234
Win Rate:             58.55%
Max Drawdown [%]:     -27.37
Avg Trade Duration:   0 days 13:00:00
Best Trade [%]:       12.16
Worst Trade [%]:      -5.29


/Users/guilistocco/Trabalho_Final/trading_strategy/trading_env/lib/python3.10/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p11742', ...)